In [2]:
# импорт библиотек
import cv2
import numpy as np
import pandas as pd
import pytesseract
import easyocr
import re
import os
import time

from PIL import Image

from IPython.display import display

## Функции

### Функция вырезания таблицы из изображения `crop_img`

In [3]:
# функция для вырезания таблицы из изображения
def crop_img(img_path, output_path):
    # Считывание изображения
    img = Image.open(img_path)
    img = np.array(img)
    if img.ndim == 3 and img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Присвоение изображению порогового значения в виде двоичного изображения
    img_bin = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 21, 10)
    
    # Обнаружение контуров
    contours, _ = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Определение самого большого контура
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Обрезание по этому контуру
    x, y, w, h = cv2.boundingRect(largest_contour)
    cropped_image = img[y:y+h, x:x+w]
    
    # Сохранение
    cv2.imwrite(output_path, cropped_image)

    return cropped_image

### Функция распознавания `img2table`

In [14]:
# функция для распознавания таблиц с изображения
# pytesseract для текста + EasyOCR для недостающих (digit=0)
# или + EasyOCR для цифр (digit=1)
def img2table(img_path, output_path, digit=1):
    # Считывание изображения
    #img = Image.open(img_path)
    #img = np.array(img)
    img = img_path
    if img.ndim == 3 and img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Присвоение изображению порогового значения в виде двоичного изображения
    img_bin = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 21, 10)

    # Ширина ядра как 100-я часть общей ширины
    kernel_len = np.array(img).shape[1] // 100

    # Определение вертикального и горизонтального ядер
    ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))
    hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))

    # Ядро размером 2x2
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))

    # Обнаружение вертикальных и горизонтальных линий
    image_1 = cv2.erode(img_bin, ver_kernel, iterations=3)
    vertical_lines = cv2.dilate(image_1, ver_kernel, iterations=3)

    image_2 = cv2.erode(img_bin, hor_kernel, iterations=3)
    horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=3)

    # Объединение горизонтальных и вертикальных линий в новом изображении
    img_vh = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)

    # Размывание и установление порогового значения
    img_vh = cv2.erode(~img_vh, kernel, iterations=2)
    _, img_vh = cv2.threshold(img_vh, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Обнаружение контуров
    contours, _ = cv2.findContours(img_vh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Преобразование tuple в list
    contours = list(contours)
    contours.reverse()

    # Подсчёт вертикальных линий (столбцов)
    num_vertical_lines = 1
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        
        if w > 20 and h > 20 and w < 500 and h < 500:
            num_vertical_lines = int(img.shape[1] / w)
            break
    
    texts = []
    # Итерация по каждому контуру (ячейке)
    for contour in contours:
        
        # Получение координат ограничивающего прямоугольника вокруг контура
        x, y, w, h = cv2.boundingRect(contour)
        if w > 20 and h > 20 and w < 500 and h < 500:
            cell_image = img[y:y + h, x:x + w]
        
            # Получение текста с ячейки
            text = pytesseract.image_to_string(cell_image, lang='rus')

            if digit == 0: # вариант "EasyOCR для недостающих"
                if text == '':
                    results = reader.readtext(cell_image)
                    if len(results) > 0:
                        text = results[0][1]
            else: # вариант "EasyOCR для цифр"
                if text == '' or re.search(r'\d', text):
                    results = reader.readtext(cell_image)
                    if len(results) > 0:
                        text = results[0][1]

            # Добавление текста в список
            texts.append(text.replace('|', '').replace('_', '').replace(',', '.'))
    
    # Исключение пустых строк вначале списка
    for text in texts[num_vertical_lines:]:
        if text == '':
            texts.remove(text)
        else:
            break
        
    # Разделение списка по количеству столбцов
    texts = [texts[i:i+num_vertical_lines] for i in range(0, len(texts), num_vertical_lines)]
    
    # Создание датафрейма
    try:
        df = pd.DataFrame(data=texts[1:], columns=texts[0])
        df.dropna(inplace=True)
        df = df.applymap(lambda x: x.replace('\n', ''))
        df = df.rename(columns=lambda x: re.sub(r'\s+', ' ', x.replace('\n', ' ')))
    except:
        df = pd.DataFrame()
    
    # Сохранение
    df.to_csv(output_path, index=False)

    return df

### Функция сравнения результатов `compare_tables`

In [5]:
# функция сравнения результатов распознавания с реальными значениями
# df1 - датасет с реальными значениями
# df2 - датасет с результатами распознавания
def compare_tables(df1, df2):
    # считаем, что правильной является 1 таблица
    
    num_matching_cells = (df1 == df2).sum().sum()

    total_cells = df1.size
    
    accuracy = num_matching_cells/total_cells
    return num_matching_cells, total_cells, accuracy

## Основная программа

In [6]:
# активация pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'

In [7]:
# активация easyocr
reader = easyocr.Reader(['ru'])

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
c:\Users\Victor\anaconda3\lib\site-packages\torchvision\models\_utils.py:252: UserWarning: Accessing the model URLs via the internal dictionary of the module is deprecated since 0.13 and will be removed in 0.15. Please access them via the appropriate Weights Enum instead.
  warnings.warn(
c:\Users\Victor\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\Victor\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [15]:
%%time
# формирование списков изображений и тестовых csv, находящихся в подпапке test
path_file = r'test\\'
files_jpg = []
files_csv = []
for file in sorted(os.listdir(path_file)):
    if file.endswith('.jpg'):
        files_jpg.append(file)
    elif file.endswith('.csv'):
        files_csv.append(file)

for i in range(len(files_jpg)):
    # засекаем время
    start_time = time.time()
    # вызов функции вырезания таблицы
    tabl = crop_img(path_file+files_jpg[i], path_file+files_jpg[i][:-4]+'_tabl.jpg')
    # вызов функции распознавания
    df = img2table(tabl, path_file+files_jpg[i][:-4]+'_rezult.csv')
    # подсчет времени выполнения
    time_fit = time.time() - start_time
    # отображение распознанной информации
    print(f'{i+1}. Результат распознавания файла {files_jpg[i]}:')
    display(df)
    print(f'Время распознавания файла {files_jpg[i]}: {time_fit:0.1f} сек. ({time.strftime("%H:%M:%S", time.gmtime(time_fit))})\n')
    # вызов функции обработки проверочного csv

1. Результат распознавания файла 141899 .jpg:


,Длка
0,плл90
1,уон7 л
2,
3,ЗИ кадемей
4,дву@
5,861[
6,дети
7,{8


Время распознавания файла 141899 .jpg: 195.4 сек. (00:03:15)

2. Результат распознавания файла 204119 .jpg:


,Дата,ДоН ва,Кол-во,Дата,Вид дон-ва,Кол-во,5,
0,8,,11.10.2017,крИд (бв),16.10.2018,кр/д (бв),450,крАд (бв)


Время распознавания файла 204119 .jpg: 25.2 сек. (00:00:25)

3. Результат распознавания файла 213950.jpg:


...

Время распознавания файла 254586 .jpg: 11.6 сек. (00:00:11)

14. Результат распознавания файла 256578 .jpg:


,810@
0,Дошрче о нвю тя
1,979{0
2,
3,
4,же
5,п2
6,
7,Н
8,дооов.
9,


Время распознавания файла 256578 .jpg: 95.0 сек. (00:01:34)

15. Результат распознавания файла 256838.jpg:


""


Время распознавания файла 256838.jpg: 22.2 сек. (00:00:22)

CPU times: total: 1h 28min 8s
Wall time: 22min 37s


### Дальше можно не смотреть, это остатки предыдущего `main` для 1 изображения

In [26]:
# вызов функции вырезания таблицы из изображения
crop_img(img_path, tabl_temp)

# вызов функции распознавания (возвращает датафрейм df)
df = img2table('tabl_temp.jpg', easy_dig_path)

# отображение распознанной информации
print('Распознанная информация:')
display(df)

Распознанная информация:


,Дата,Вид дон-ва,Кол-во,Дата,Вид дон-ва,Кол-во,Дата,Вид дон-ва,Кол-во
0,14.02.2006,кр/д (бв),420,15.07.2016,кр/д (бв),450,04.08.2018,кр/д (бв),450
1,11.06.2014,кр/д (бв),350,11.10.2016,кр/д (бв),450,26.12.2018,кр/д (бв),450
2,30.10.2014,кр/д (бв),450,21.12.2016,кр/д (бв),450,29.03.2019,кр/д (бв),450
3,13.08.2015,кр/д (бв),450,21.06.2017,кр/д (бв),450,11.10.2022,кр/д (бв),450
4,30.10.2015,кр/д (бв),450,,,,,,


In [27]:
# загрузка и отображение проверочной информации
df_test = pd.read_csv('tablica1.csv', header=0, dtype = 'str')
print('Проверочная информация:')
display(df_test)

Проверочная информация:


,Дата,Вид дон-ва,Количество,Дата.1,Вид дон-ва.1,Количество.1,Дата.2,Вид дон-ва.2,Количество.2
0,1,2,3,4,5,6,7,8,9
1,14.02.2006,кр/д (бв),420,15.07.2016,кр/д (бв),450,04.08.2018,кр/д (бв),450
2,11.06.2014,кр/д (бв),350,11.10.2016,кр/д (бв),450,26.12.2018,кр/д (бв),450
3,30.10.2014,кр/д (бв),450,21.12.2016,кр/д (бв),450,29.03.2019,кр/д (бв),450
4,13.08.2015,кр/д (бв),450,21.06.2017,кр/д (бв),450,11.10.2022,кр/д (бв),450
5,30.10.2015,кр/д (бв),450,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# замена NaN, удаление первой строки с номерами столбцов и реиндексация в проверочном датасете
df_test.fillna('', inplace=True)
df_test = df_test.drop(labels=0, axis = 0).reset_index(drop=True)

# переименование столбцов в распознанном и проверочном датафреймах, чтобы совпадали
df.columns = [str(x) for x in list(range(9))]
df_test.columns = [str(x) for x in list(range(9))]

# удаление пробелов в ячейках
df = df.applymap(lambda x: x.replace(' ', ''))
df_test = df_test.applymap(lambda x: x.replace(' ', ''))

# вызов функции сравнения результата распознавания
num_matching_cells, total_cells, accuracy = compare_tables(df_test, df)

# вывод результата сверки
print(f'Корректно распознанных ячеек: {accuracy*100:0.2f} %.')

Корректно распознанных ячеек: 100.00 %.
